# Sure we have enable GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# Install library and ready dataset

In [1]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle/ && cp /content/drive/MyDrive/kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download ee4211-object-detection --force
!unzip /content/ee4211-object-detection.zip -d /content

In [ ]:
# Our dataset that process with yolov5 label format
!unzip /content/drive/MyDrive/yoloUseEEv2.zip -d /content

# Install yolov5

In [4]:
!git clone https://github.com/ultralytics/yolov5  
%cd yolov5
%pip install -qr requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5
     |████████████████████████████████| 596 kB 34.8 MB/s 


In [6]:
import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.1-118-g3bb233a torch 1.10.0+cu111 CUDA:0 (Tesla T4, 15110MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 40.3/166.8 GB disk)


# Training

In [ ]:
!python train.py --img 640 --batch 16 --epochs 140 --data /content/yoloUseEEv2/flow.ymal --weights yolov5l.pt --cache

# Download model

In [ ]:
!zip -r /content/yolov5Train4.zip /content/yolov5/runs/train/exp2

# Generating csv

In [ ]:
import torch
import os
import pandas as pd
import numpy as np 
import PIL

# model location
model = torch.hub.load('ultralytics/yolov5', 'custom', path="/content/yolov5/runs/train/exp2/weights/best.pt")

imgDir = "/content/Data/Test/images"
imgArr = os.listdir(imgDir)

# imgArr to proper img with path
imgs = [os.path.join(imgDir,i) for i in imgArr]

model.conf = 0.20  # NMS confidence threshold
model.iou = 0.30  # NMS IoU threshold

# transfer bbox to coco format
def calBoxPt (xmin,ymin, xmax, ymax, width, height):
    x = xmin / width
    y = ymin / height

    w = (xmax - xmin) / width
    h = (ymax - ymin) / height

    return [x,y,w,h]

finalArr = []
for ind, img in enumerate(imgs):
    image = PIL.Image.open(img)
    width, height = image.size

    results = model(img) # predict result
    res = results.pandas().xyxy[0] # predict result to bbox arr

    if len(res.confidence) >= 1 : # have predict result box
      finalArr.append([ imgArr[ind] , int(res.xmin[0]), int(res.ymin[0]), int(res.xmax[0]), int(res.ymax[0])  ])
    else:
      finalArr.append([ imgArr[ind], 0,0,0,0 ])


print(len(finalArr))
finalArr = np.array(finalArr)
df = pd.DataFrame({
    'ImageId': finalArr[:, 0],
    ' X_min': finalArr[:, 1],
    ' Y_min': finalArr[:, 2],
    ' X_max': finalArr[:, 3],
    ' Y_max': finalArr[:, 4]
})

df.to_csv("outputNewSmall.csv" , index=False)